In [1]:
from new_helper import *
from super_helper import *

In [17]:
id = "06535"
DOMAIN = "test"

project_id = f"{id}@{DOMAIN}"

directory_path = "upload"

print(40*"-")



----------------------------------------


In [3]:


schema = get_schema() #super_helper.py

print(schema)

processed_data = process_files(directory_path, project_id, schema) #new_helper.py
 
    

<Schema: ['abstract', 'created', 'creator', 'date', 'description', 'file_cid', 'format', 'full_text', 'language', 'metadata_cid', 'modified', 'project_id', 'publisher', 'subject', 'title']>
file path:  upload/COVID19-MLSF--A-multi-task-learning-based-stock-market_2023_Expert-Systems-w.pdf
file name:  COVID19-MLSF--A-multi-task-learning-based-stock-market_2023_Expert-Systems-w.pdf
file cid:  QmUq29KRwpTdvScB5oYzEDobDHyb4N1f9eaZXm4VaMCgiW
file metadata cid:  Qme5cwgyMn5R2LRqT6jvrg4GfjuauPKwGR4m6njHpzhAtR
file_key : file_1
joined_cids : QmUq29KRwpTdvScB5oYzEDobDHyb4N1f9eaZXm4VaMCgiW, Qme5cwgyMn5R2LRqT6jvrg4GfjuauPKwGR4m6njHpzhAtR
	Entering "create_contract"
('STATELESS_VALIDATION_SUCCESS', 1, 0)
('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
('STATEFUL_VALIDATION_SUCCESS', 3, 0)
('COMMITTED', 5, 0)
	Leaving "create_contract"
	Entering "get_engine_receipts_address"
	Leaving "get_engine_receipts_address"
	Entering "set_account_detail"
None
('STATELESS_VALIDATION_SUCCESS', 1, 0)
('ENOUGH_SIGNATURES_C

2025-01-09 15:15:49.335 | INFO     | super_helper:setup_index:328 - Opened existing index.
2025-01-09 15:15:49.473 | INFO     | super_helper:index_metadata:186 - Metadata indexed successfully: Qme5cwgyMn5R2LRqT6jvrg4GfjuauPKwGR4m6njHpzhAtR


('STATEFUL_VALIDATION_SUCCESS', 3, 0)
('COMMITTED', 5, 0)
	Leaving "set_account_detail"
hash : b'e2659cd178643e10ff1d4d5174147250e7313b453bb68649fd41678be7893ac6'
This is ix:  FileIndex(FileStorage('indexdir'), 'MAIN')


In [4]:
account_detail = get_account_detail(project_id)
print(project_id, account_detail)


06535@test { "admin@test" : { "file_1" : "QmUq29KRwpTdvScB5oYzEDobDHyb4N1f9eaZXm4VaMCgiW, Qme5cwgyMn5R2LRqT6jvrg4GfjuauPKwGR4m6njHpzhAtR", "file_2" : "QmUq29KRwpTdvScB5oYzEDobDHyb4N1f9eaZXm4VaMCgiW, Qmd1NaZKsfeF47LMX7a7DgkDUxUYi2ykQpKTbqYeVaVw62", "file_3" : "QmZnh5Uuo7moZKMESFHpRjQFbbND5e3nArCzZg8uLARAgg, QmXgwRkPFxG2FWQTbVGgKsaEz7thbg4WLBcndsRfKUfeaa", "file_4" : "QmUq29KRwpTdvScB5oYzEDobDHyb4N1f9eaZXm4VaMCgiW, QmbKquQj5bMdMMHaNfacbXkT9NgwWbj157EPpRVi1tPPdx", "file_5" : "QmV1vYkuiwKGPA5Tyi4ECNFfWYocyTtQ3E5j7CSvPBX49Q, QmY7xwK4beVPQLbeKxEAeStqxY69f7GzRhqTukXXsLMkMh", "linked_user" : "crazy_nash@test", "project_metadata_cid" : "QmbmyecKUbwCDFPt6Uddx411bCvefLiRXPxHs5vnYz4XZm" } }


In [5]:
# recreate_index() #Controls the reset of the index manually

In [6]:
from loguru import logger
import json
from whoosh.index import open_dir

# Open an existing index (adjust the path to your index directory)
index_path = "indexdir"
index = open_dir(index_path)

# Directory for file downloads
download_path = "download"

# Perform a keyword search
keyword = "must"
search_results, project_ids_with_cids = search_index(index, keyword)

# Check if there are no search results
if not search_results:
    logger.warning(f"No search results found for keyword: '{keyword}'. Exiting the script.")
else:
    # Process each dictionary in search results
    for result_dict in search_results:
        project_id = result_dict.get('project_id')
        file_cid = result_dict.get('file_cid')
        metadata_cid = result_dict.get('metadata_cid')

        if not project_id or not file_cid or not metadata_cid:
            logger.error(f"Missing required data in result: {result_dict}")
            continue

        # Log the retrieved project details
        logger.info(f"Processing Project ID: {project_id}")
        logger.debug(f"File CID: {file_cid}")
        logger.debug(f"Metadata CID: {metadata_cid}")
        # file_metadata_json = download_json_from_ipfs(metadata_cid)
        # print("file_metadata_json:", file_metadata_json)
        

        # Fetch project details from the blockchain
        project_details = get_account_detail(project_id)
        if not project_details:
            logger.error(f"No project details found for Project ID: {project_id}.")
            continue

        logger.debug(f"Fetched project details for {project_id}: {project_details}")

        # Parse blockchain data
        try:
            blockchain_data = json.loads(project_details)
        except json.JSONDecodeError as e:
            logger.error(f"Error decoding project details JSON for {project_id}: {e}")
            continue

        # Validate file CID and fetch project details
        validation_result = fetch_project_details(file_cid, blockchain_data)
        if validation_result["is_valid"]:
            project_metadata_cid = validation_result.get("project_metadata_cid")
            linked_user = validation_result.get("linked_user")
            file_metadata_cid = validation_result.get("metadata_cid")
            
            # download_file(file_metadata_json, download_path, project_id, file_cid)

            logger.info(f"Valid File CID for {project_id}.")
            logger.debug(f"Project Metadata CID: {project_metadata_cid}")
            logger.debug(f"Linked User: {linked_user}")

            # Fetch and process metadata and user details
            if project_metadata_cid:
                logger.debug(f"Processing project metadata CID: {project_metadata_cid}")
                project_metadata = download_json_from_ipfs(project_metadata_cid)
                logger.debug(f"Downloaded project metadata: {project_metadata}")

            if linked_user:
                logger.debug(f"Processing linked user: {linked_user}")
                user_details = get_account_detail(linked_user)
                try:
                    user_details = json.loads(user_details)
                except json.JSONDecodeError as e:
                    logger.error(f"Error decoding user details JSON for {linked_user}: {e}")
                    continue

                user_json_ld_cid = user_details.get("admin@test", {}).get("user_json_ld_cid", None)
                if user_json_ld_cid:
                    logger.info(f"User JSON-LD CID: {user_json_ld_cid}")
                    user_metadata = download_json_from_ipfs(user_json_ld_cid)
                    logger.debug(f"Downloaded user metadata: {user_metadata}")
                else:
                    logger.warning(f"User JSON-LD CID not found for linked user {linked_user}.")
            
            if metadata_cid:
                logger.info(f"Processing metadata CID: {metadata_cid}")
                file_metadata = download_json_from_ipfs(metadata_cid)
                file_metadata_json = download_file(file_metadata, download_path, project_id, file_cid)
                logger.info(f"Downloaded file metadata: {metadata_cid}")
                logger.info(f"file metadata: {file_metadata}")
                logger.info(file_metadata_json)
            
        
        else:
            logger.warning(f"Invalid File CID for Project ID: {project_id}. Skipping metadata processing.")


2025-01-09 15:15:49.582 | INFO     | super_helper:search_index:204 - Starting keyword search...
2025-01-09 15:15:49.584 | INFO     | super_helper:search_index:205 - Keyword: 'must'
2025-01-09 15:15:49.612 | INFO     | super_helper:search_index:216 - Search successful: Found 3 result(s).
2025-01-09 15:15:49.616 | INFO     | super_helper:search_index:218 - 1. Project Id: 06535@test, File CID: QmUq29KRwpTdvScB5oYzEDobDHyb4N1f9eaZXm4VaMCgiW, Metadata CID: QmdyAE132FSeSQaqUHzE5GtMrEwy821ScAYKEXsxpWSnV5, Title: covid19-mlsf: a multi-task learning-based stock market forecasting framework during the covid-19 pandemic
2025-01-09 15:15:49.619 | INFO     | super_helper:search_index:218 - 2. Project Id: 06535@test, File CID: QmUq29KRwpTdvScB5oYzEDobDHyb4N1f9eaZXm4VaMCgiW, Metadata CID: QmZxZTHgeGuQmkSwHJLcfzsPVtfvMCXckzw5EGVKY9Ecda, Title: covid19-mlsf: a multi-task learning-based stock market forecasting framework during the covid-19 pandemic
2025-01-09 15:15:49.622 | INFO     | super_helper:sear

In [7]:
# Perform a keyword search
# keyword = "paper"
# search_results, project_ids_with_cids = search_index(index, keyword)

# if not search_results:
#     logger.warning(f"No search results found for keyword: '{keyword}'. Exiting the script.")
# else:
#     for result in search_results:
#         process_search_result(result)